## Part 1

In [1]:
import numpy as np
from pprint import pprint

In [2]:
def read_train_file(filename):
    with open(filename, encoding='utf-8') as f:
        file_content = f.read()

    # Split the entire file into sentences. Output: List of sentences
    sentences = file_content.strip().split('\n\n')

    # Split each sentence into their token_tag pair
    # Output: List of sentences. Each sentence is a list of token_tag_pair
    token_tag_pairs = [i.split('\n') for i in sentences]

    # Separate each token_tag_pair into a list of [token, tag].
    # Output: [[[token, tag], [token, tag], ...], [[token, tag], [token, tag], ...], ...]
    for idx, sentence in enumerate(token_tag_pairs):
        token_tags = [i.rsplit(' ', maxsplit=1) for i in sentence]
        token_tag_pairs[idx] = token_tags

    return token_tag_pairs

In [3]:
train_dataset = './dataset/train'

In [4]:
train_dataset = read_train_file(train_dataset)

In [5]:
emission_count = {}
transition_count = {}
state_count = {}
possible_states = []

transition_count['START'] = {}
for sentence in train_dataset:
    prev_state = None
    
    for token, tag in sentence:
        if tag not in possible_states:
            possible_states.append(tag)

        if emission_count.get(token) == None:
            emission_count[token] = {}
        
        emission_count[token][tag] = emission_count[token].get(tag, 0) + 1

        if prev_state != None:
            if transition_count.get(prev_state) == None:
                transition_count[prev_state] = {}
            transition_count[prev_state][tag] = transition_count[prev_state].get(tag, 0) + 1

        else:
            transition_count['START'][tag] = transition_count['START'].get(tag, 0) + 1
            state_count['START'] = state_count.get('START', 0) + 1

        state_count[tag] = state_count.get(tag, 0) + 1
        prev_state = tag

    transition_count[prev_state]['STOP'] = transition_count[prev_state].get('STOP', 0) + 1

In [6]:
pprint(emission_count)
pprint(transition_count)
pprint(state_count)
pprint(possible_states)

{'!': {'O': 280},
 '"': {'B-negative': 1, 'I-negative': 1, 'O': 47},
 '#': {'O': 1},
 '$': {'O': 46},
 '%': {'O': 3},
 '&': {'O': 12},
 "'": {'I-negative': 1, 'O': 18},
 '(': {'I-positive': 9, 'O': 78},
 ')': {'I-positive': 8, 'O': 84},
 '*': {'O': 4},
 '+': {'I-positive': 1, 'O': 4},
 ',': {'I-positive': 3, 'O': 962},
 '-': {'O': 85},
 '.': {'O': 1508},
 '..': {'O': 13},
 '...': {'O': 99},
 '/': {'I-positive': 2, 'O': 14},
 '0': {'O': 2},
 '1': {'O': 8},
 '1/2': {'O': 3},
 '1/26': {'O': 1},
 '10': {'O': 6},
 '10-15': {'O': 2},
 '100': {'O': 4},
 '11': {'O': 1},
 '12': {'O': 2},
 '13': {'O': 1},
 '14': {'O': 2},
 '15': {'O': 6},
 '17': {'O': 1},
 '170': {'O': 1},
 '18': {'O': 1},
 '1st': {'O': 3},
 '2': {'O': 15},
 '20': {'O': 8},
 '2002': {'O': 1},
 '23rd': {'O': 1},
 '24': {'O': 2},
 '25': {'O': 1},
 '27': {'O': 1},
 '29': {'O': 1},
 '2nd': {'O': 3},
 '2times': {'O': 1},
 '3': {'I-positive': 1, 'O': 10},
 '3-4': {'O': 1},
 '3-6': {'O': 1},
 '30': {'O': 7},
 '30-70': {'O': 1},
 '32nd'

In [7]:
f = {}

for token, tags in emission_count.items():
    for tag, e_count in tags.items():
        key = "emission: " + tag + '+' + token
        e_prob = np.log(e_count/state_count[tag])
        f[key] = e_prob

for prev_tag, next_tags in transition_count.items():
    for next_tag, t_count in next_tags.items():
        key = "transition: " + prev_tag + '+' + next_tag
        t_prob = np.log(t_count/state_count[prev_tag])
        f[key] = t_prob

pprint(f)

{'emission: B-negative+"': -5.934894195619588,
 'emission: B-negative+BFC': -5.934894195619588,
 'emission: B-negative+Bark': -5.934894195619588,
 'emission: B-negative+Caesar': -5.934894195619588,
 'emission: B-negative+Casa': -5.241747015059643,
 'emission: B-negative+Chilli': -5.934894195619588,
 'emission: B-negative+Decor': -5.934894195619588,
 'emission: B-negative+Delivery': -5.241747015059643,
 'emission: B-negative+Dessert': -5.934894195619588,
 'emission: B-negative+Drinks': -5.934894195619588,
 'emission: B-negative+French': -5.934894195619588,
 'emission: B-negative+Go': -5.934894195619588,
 'emission: B-negative+Haru': -5.934894195619588,
 'emission: B-negative+Korean': -5.934894195619588,
 'emission: B-negative+Leon': -5.934894195619588,
 'emission: B-negative+Management': -5.934894195619588,
 'emission: B-negative+Mioposto': -5.934894195619588,
 'emission: B-negative+PIZZA': -5.934894195619588,
 'emission: B-negative+PLACE': -5.241747015059643,
 'emission: B-negative+Piz

## Part 2

In [8]:
'''
calculate_score(x,y):
Helps to calulate the score for a given pair of input and output sequence pair (x,y)
Based on 2 features, emission and transition

Parameters:
x: List of tokens, e.g. x = x1, x2, ..., xn     Type: list[str]
y: List of tokens, e.g. y = y1, y2, ..., yn     Type: list[str]
f: Dictionary of feature weights                   Type: Dict{features: weights}
'''

def calculate_score(x,y,f):
    assert len(x) == len(y)

    feature_count = {}

    prev_tag = 'START'
    score = 0

    length = len(x)
    for i in range(length):
        e_key = "emission: " + y[i] + '+' + x[i]
        t_key = "transition: " + prev_tag + '+' + y[i]

        if e_key in f.keys():
            feature_count[e_key] = feature_count.get(e_key, 0) + 1

        if t_key in f.keys():
            feature_count[t_key] = feature_count.get(t_key, 0) + 1

        prev_tag = y[i]
        
    t_key = "transition: " + prev_tag + '+' + 'STOP'
    if t_key in f.keys():
            feature_count[t_key] = feature_count.get(t_key, 0) + 1

    for feature, count in feature_count.items():
        score += f[feature] * count

    return score

In [9]:
def viterbi(sentence, f):
        # BASE CASE
        scores = {
            0: {
                'START' : 0
            }
        }

        index = 1

        # Forward Algorithm - From START to index N
        for token in sentence:
            scores[index] = {}

            for state in possible_states:
                state_scores = {}

                for prev_tag in scores[index-1].keys():
                    e_key = "emission: " + state + '+' + token
                    t_key = "transition: " + prev_tag + '+' + token
                    e_prob = f.get(e_key, 0)
                    t_prob = f.get(t_key, 0)

                    # t_prob = self._calculate_transition_MLE(prev_tag, state)
                    # e_prob = self._calculate_emission_MLE_UNK(token, state)

                    if t_prob > 0 and e_prob > 0:
                        state_scores[prev_tag] = \
                            scores[index-1][prev_tag] + \
                            np.log(t_prob) + \
                            np.log(e_prob)
                    else:
                        state_scores[prev_tag] = float('-inf')

                best_score = max(state_scores.values())
                scores[index][state] = best_score

            index += 1

        # Forward Algorithm - From index N to STOP
        state_scores = {}
        for prev_tag in scores[index-1].keys():
            t_key = "transition: " + prev_tag + '+' + 'STOP'
            t_prob = f.get(t_key, 0)
            if t_prob > 0:
                state_scores[prev_tag] = scores[index-1][prev_tag] + np.log(t_prob)
            else:
                state_scores[prev_tag] = float('-inf')

        y_n = max(state_scores, key=state_scores.get)
        prediction_reversed = [y_n]

        # Backtracking Algorithm
        for n in reversed(range(1,index)):
            state_scores = {}

            for state in scores[n-1].keys():
                t_key = "transition: " + state + '+' + prediction_reversed[-1]
                t_prob = f.get(t_key, 0)
                
                # t_prob = self._calculate_transition_MLE(state, prediction_reversed[-1])

                if t_prob > 0:
                    state_scores[state] = scores[n-1][state] + np.log(t_prob)

            if all(prob == float('-inf') for prob in state_scores.values()):
                prediction_reversed.append('O')
            else:
                best_state = max(state_scores, key=state_scores.get)
                prediction_reversed.append(best_state)

        prediction = []
        prediction_reversed.reverse()

        for idx, token in enumerate(sentence):
            prediction.append([token, prediction_reversed[idx+1]])

        return prediction

In [10]:
viterbi(["Loved","it"], f)

[['Loved', 'O'], ['it', 'O']]